---
## Josh Knize and Pradyumn Pathak Final Project Submission
---

### Project Setup:

We would recommend creating a virtual environment for this notebook:
* Recomended Python=3.8

In [ ]:
!git clone https://github.com/facebookresearch/detectron2.git
!python -m pip install -e detectron2

!pip install gdown
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install fvcore
!pip install opencv-python

!gdown --id 1MM6kODxnDvkVzzOuOYYbwrP5uou1DOC -O datasets.zip
!gdown --id 13JG4Sh7Fpad8O6eB_fQe7C2wQmO7hRtj -O Hebbnet_backbone.zip

!python -c "import zipfile; zipfile.ZipFile('dataset.zip', 'r').extractall('dataset')"
!python -c "import zipfile; zipfile.ZipFile('Hebbnet_backbone.zip', 'r').extractall('Hebbnet_backbone')"
!python -c "import shutil; shutil.move('Hebbnet_backbone/__init__.py', './detectron2/detectron2/modeling/backbone/__init__.py'); shutil.move('Hebbnet_backbone/hebbnet_backbone.py', './detectron2/detectron2/modeling/backbone/hebbnet_backbone.py'); shutil.move('Hebbnet_backbone/hebbnet_backbone.yaml', './detectron2/configs/COCO-Detection/hebbnet_backbone.yaml')"


## !! After the above stage please move this notebook inside the `detectron2` directory, and set the following variable to the same path:

In [ ]:
PATH_detectron2 = "/path/to/your/notebook"

### Details about the dataset:

* This is a custom dataset created from COCO-Dataset
* It is a detection dataset only containing one object - `dog`
* TODO: ADD MORE INFO ABOUT THE DATASET

In [ ]:
# Setting up runtime Work Directory and Importing Libraries

import os
os.chdir(PATH_detectron2)
print(os.getcwd())
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog

from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
import torch

from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

In [ ]:
# Setting up the custom dataset for training:

register_coco_instances("coco_train_dog", {}, "../datasets/coco/annotations/dog_instances_train2017.json", "../datasets/coco/train2017_dog")
register_coco_instances("coco_val_dog", {}, "../datasets/coco/annotations/dog_instances_val2017.json", "../datasets/coco/val2017_dog")

my_dataset_metadata = MetadataCatalog.get("coco_train_dog")
my_dataset_metadata.thing_classes = ["dog"]
dataset_dicts = DatasetCatalog.get("coco_train_dog")


---
## 1. The following segment is for Training and Evaluating The FRCNN with Resnet Backbone for control:

In [ ]:
# 1a. Set the configuration to load teh Resent Based FRCNN architecture:

cfg = get_cfg()
cfg.merge_from_file("configs/COCO-Detection/faster_rcnn_R_50_C4_1x.yaml") #ImageNet pre-trained
cfg.OUTPUT_DIR = f"{PATH_detectron2}/output/dog_resnet_test"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7

cfg.DATASETS.TRAIN = ("coco_train_dog",)
cfg.DATASETS.TEST = ("coco_val_dog",)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.5
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0025
cfg.SOLVER.MAX_ITER = 50000
cfg.SOLVER.CHECKPOINT_PERIOD = 1000
cfg.TEST.EVAL_PERIOD = 1000

# run on GPU
cfg.MODEL.DEVICE = 'cuda'

In [ ]:
# 1b. Load the Trainer for the defiend Resnet-FRCNN architecture and train:

trainer = DefaultTrainer(cfg)
trainer.model.to(cfg.MODEL.DEVICE)

trainer.train()

In [ ]:
# 1c. Saving and Evaluation of the model

output_dir = f"{PATH_detectron2}/dog_resnet_test"
cfg.MODEL.WEIGHTS = output_dir + '/RESNET_BackBone_model_final.pth'

trainer.model.eval()

evaluator = COCOEvaluator("coco_val_subset", ("bbox",), False, output_dir=output_dir)
val_loader = build_detection_test_loader(cfg, "coco_val_dog")
print(inference_on_dataset(trainer.model, val_loader, evaluator))

### Importand Note:
* The FRCNN with Resnet Backbone uses **pretrained weights** on ImageNet but our HebbNet architecture is learning from scrach in this detection architecture
* The Resnet architecture in Resnet Backbone has a larger input `img-size`, allowing for higher feature throughput from the get go
---

---
## 2. Training the FRCNN with Hebbnet Backbone:

In [ ]:
# 2a. Setting the primary configurations, ensuring we use the [Custom] HebbNet Backbone:

cfg = get_cfg()
cfg.merge_from_file("configs/COCO-Detection/hebbnet_backbone.yaml")     # Loading the Custom HebbNet Backbone Configuration template
cfg.OUTPUT_DIR = f"{PATH_detectron2}/output/dog_hebbnet_test"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7

cfg.DATASETS.TRAIN = ("coco_train_dog",)
cfg.DATASETS.TEST = ("coco_val_dog",)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.OUTPUT_LAYER_SIZE = 1
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.5
cfg.SOLVER.IMS_PER_BATCH = 1
cfg.SOLVER.BASE_LR = .00002
cfg.SOLVER.MAX_ITER = 5500*2

# run on GPU
cfg.MODEL.DEVICE = 'cuda'

#### While Loading the secondary custom configurations:
- Our input `image-size` is `128x128`, The reason being we cannot train the model for larger `image-size` due to the following problem:
    - We run into `CUDA: Out of Memory Model`: Due to the hidden layers having way too many neurons caused due to a lack of Convolution
- We are also leaving out `PROPOSAL_GENERATOR` from our model
- The Batch size is set to 1, because we are using Hebbian Learning, but also because of memory constraints

In [ ]:
# Loading Secondary configs for custom backbone:

cfg.INPUT.MIN_SIZE_TRAIN = (128,)
cfg.INPUT.MIN_SIZE_TRAIN_SAMPLING = "choice"
cfg.INPUT.MAX_SIZE_TRAIN = 128
cfg.INPUT.MIN_SIZE_TEST = 128
cfg.INPUT.MAX_SIZE_TEST = 128
# cfg.PROPOSAL_GENERATOR: PrecomputedProposals # this may be an option to potentially avoid issues with proposal generation
cfg.MODEL.ROI_HEADS.IN_FEATURES: ['res4']


- There are also major changes to the `ROIHead` and `BoXHead` in FRCNN:
    - We opted to using the `StandardROIHeads` because it made the integration of HebbNet Backbone easier
    - We are also limited the `Number Of Channels` in the `FilterMaps` that the `ROIHead` can work with to `8` due to GPU memory constraints
    - The `Number Of Fully-Connected Layers` in the `FilterMaps` is also `8` due to GPU memory constraints

In [ ]:
cfg.MODEL.ROI_HEADS.NAME = "StandardROIHeads"
cfg.MODEL.ROI_BOX_HEAD.NAME = "FastRCNNConvFCHead"
cfg.MODEL.ROI_BOX_HEAD.FC_DIM = 8           # Fully Connected Channel Depth to 8
cfg.MODEL.ROI_BOX_HEAD.CONV_DIM = 8         # Channel Depth to 8
cfg.MODEL.ROI_BOX_HEAD.NUM_CONV = 2
cfg.MODEL.ROI_BOX_HEAD.NUM_FC = 2
cfg.SOLVER.CHECKPOINT_PERIOD = 9999999999999999999999999999999999

#### Finally, the following are the additional parameters exclusive to the Hebbian Backbone:

In [ ]:
cfg.MODEL.NUM_HIDDEN = 8
cfg.MODEL.HEBB_LR = .00000001 # fewest number of zeros "allowed" before gradient explosion

cfg.MODEL.IMG_VIS = False
cfg.MODEL.FEAT_VIS = False
cfg.MODEL.FEAT_VIS_NUM = 0

In [ ]:
# 2b. Load the Trainer for the defiend Resnet-FRCNN architecture and train:

trainer = DefaultTrainer(cfg)
trainer.model.to(cfg.MODEL.DEVICE)

trainer.train()

In [ ]:
# 2c. Saving and Evaluation of the model

output_dir = f"{PATH_detectron2}/dog_hebbnet_test"
cfg.MODEL.WEIGHTS = output_dir + '/HebbNet_BackBone_model_final.pth'

trainer.model.eval()

evaluator = COCOEvaluator("coco_val_subset", ("bbox",), False, output_dir=output_dir)
val_loader = build_detection_test_loader(cfg, "coco_val_dog")
print(inference_on_dataset(trainer.model, val_loader, evaluator))